In [1]:
import numpy as np
from matplotlib import pyplot as plt

class player():
    def __init__(self, player_sum, dealer_card, usable_ace):
        self.player_sum = player_sum
        self.dealer_card = dealer_card
        self.usable_ace = usable_ace
        self.using_ace = self.usable_ace
        
    def add_card_sum(self, new_card):
        if (self.using_ace == True) and (self.player_sum + new_card >21):
            self.player_sum = self.player_sum + new_card -10
            self.using_ace = False
            
        else:
            self.player_sum = self.player_sum + new_card
            
    def bust(self):
        if self.player_sum > 21:
            return True
        return False
    
    def define_state(self):
        return (self.player_sum, self.dealer_card, self.usable_ace)
    
    def should_hit(self, policy):
        return policy[self.define_state()]
    
    def player_value(self):
        return self.player_sum

In [2]:
class dealer:
    def __init__(self, faceUp_card):
        self.cards = faceUp_card
        
    def hit_card(self, card):
        self.cards.append(card)
        
    def dealer_sum(self):
        dealersum = 0
        dealer_ace = 0
        
        for card in self.cards:
            if card == 1:
                dealer_ace += 1
            else:
                dealersum += card
                
        while dealer_ace > 0:
            dealer_ace -= 1
            dealersum += 10
            
            if dealersum > 21:
                dealer_ace += 1
                dealersum -= 10
                break
        return dealersum
            
    def bust(self):
        if self.dealer_sum() > 21:
            return True
        return False
    
    def dealer_hit_policiy(self):
        if self.dealer_sum() >= 17:
            return False
        return True

In [3]:
class stateactioninfo():
    def __init__(self):
        self.state_action_pair = []
        self.state_action_map = set()
        
    def add_pair(self, pair):
        if pair in self.state_action_map:
            return 
        self.state_action_pair.append(pair)
        self.state_action_map.add(pair)
        

In [4]:
def evaluate_action_value_pair(action_val_map,  count, policy_map, state_action_pair, recent_reward):
    for pair in state_action_pair:
        count[pair] += 1
        action_val_map[pair] = action_val_map[pair] + ((recent_reward - action_val_map[pair]) / count[pair])
        
        state = pair[0]
        should_hit = False
        
        if action_val_map[(state, True)] > action_val_map[(state, False)]:
            should_hit = True
            
        policy_map[state] = should_hit        

In [5]:
def pickcard():
    card = np.random.randint(1,14)
    
    if card > 9:
        return 10
    else:
        return card

In [6]:
def initialize_simulation(action_val_map, policy_map, count):
    player_sum = np.random.randint(11, 22)
    usable_ace = bool(np.random.randint(0,2))
    dealer_faceup_card = np.random.randint(1, 11)
    
    player_object = player(player_sum, dealer_faceup_card, usable_ace)
    dealer_object = dealer([dealer_faceup_card])  #Send it as a list
    
    stateactioninfo_object = stateactioninfo()
    
    should_hit = bool(np.random.randint(0, 2))
    stateactioninfo_object.add_pair((player_object.define_state(), should_hit))
    
    if should_hit == True:
        new_card = pickcard()
        player_object.add_card_sum(new_card)
        
        while player_object.bust() == False and player_object.should_hit(policy_map) == True:
            stateactioninfo_object.add_pair((player_object.define_state(), True))
            new_card = pickcard()
            player_object.add_card_sum(new_card)
            
    if player_object.bust() == True:
        evaluate_action_value_pair(action_val_map,  count, policy_map,
                                   stateactioninfo_object.state_action_pair, -1)
        
        return
    
    stateactioninfo_object.add_pair((player_object.define_state(), False))
    
    # Dealer will start playing from this point onwards.
    
    dealer_card = pickcard()
    dealer_object.hit_card(dealer_card)
    
    while dealer_object.bust() == False and dealer_object.dealer_hit_policiy() == True:
        dealer_card = pickcard()
        dealer_object.cards.append(dealer_card)

    if dealer_object.bust() == True or dealer_object.dealer_sum() < player_object.player_value():
        evaluate_action_value_pair(action_val_map,  count, policy_map,
                                   stateactioninfo_object.state_action_pair, 1) 
    
    elif dealer_object.dealer_sum() > player_object.player_value():
        evaluate_action_value_pair(action_val_map,  count, policy_map,
                                   stateactioninfo_object.state_action_pair, -1) 
        
    else:
        evaluate_action_value_pair(action_val_map,  count, policy_map,
                                   stateactioninfo_object.state_action_pair, 0) 

In [7]:
from tqdm import tqdm
def MonteCarlo_Driver():
    action_val_map = {} # Keeps record for each State and Action. e.g. ((player_sum, dealer_sum, usable_ace), True)
    policy_map = {} # Indiactes for each state what is the BEST action.
    count = {}      # How many times a State-Action pair has been visited.
    
    for player_sum in range(11, 22):
        for usable_ace in range(0, 2):
            for dealer_card in range(1, 11):
                state = (player_sum, dealer_card, bool(usable_ace))
                action_val_map[(state, True)] = 0
                action_val_map[(state, False)] = 0
                
                count[(state, True)] = 0
                count[(state, False)] = 0
                
                if player_sum == 20 or player_sum == 21:
                    policy_map[state] = False
                    
                else:
                    policy_map[state] = True
                    
    for i in tqdm(range(5000000), desc = 'Blackjack : Monte Carlo Simulation', colour='green'):
        initialize_simulation(action_val_map, policy_map, count)
        
    return policy_map

In [8]:
q_s_a = MonteCarlo_Driver()

In [9]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

column = ['Player_Sum', 'Dealer_Card', 'Usable_Ace', 'Policy (𝛑*)']
Q_Dataframe = pd.DataFrame(columns = column)


for state, action in q_s_a.items():
    s_a = [state[0], state[1], state[2], action]
    dictionary = dict(zip(column, s_a))
    Q_Dataframe = Q_Dataframe.append(dictionary, ignore_index = True)
    
Q_Dataframe['Policy (𝛑*)'].replace({False: 'Stick', True: 'Hit'}, inplace=True)   
# Q_Dataframe.head() 

In [10]:
#Without Usable Ace
color = {'Hit' : 'green', 'Stick' : 'red'}
Q_df_without = Q_Dataframe[Q_Dataframe['Usable_Ace'] == False]

plt.scatter(Q_df_without['Dealer_Card'], Q_df_without['Player_Sum'] - 10, 
           c = Q_df_without['Policy (𝛑*)'].apply(lambda x: color[x]))


plt.xticks(range(1, 11), [ 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10' ])
plt.yticks(range(1, 12), [ '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21' ])

plt.xlabel('Dealer face-up card', color = 'blue', size = 15)
plt.ylabel('Player Sum', color = 'darkred', size = 15)

In [11]:
#With Usable Ace
color = {'Hit' : 'green', 'Stick' : 'red'}
Q_df_without = Q_Dataframe[Q_Dataframe['Usable_Ace'] == True]

plt.scatter(Q_df_without['Dealer_Card'], Q_df_without['Player_Sum'] - 10, 
           c = Q_df_without['Policy (𝛑*)'].apply(lambda x: color[x]))


plt.xticks(range(1, 11), [ 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10' ])
plt.yticks(range(1, 12), [ '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21' ])

plt.xlabel('Dealer face-up card', color = 'blue', size = 15)
plt.ylabel('Player Sum', color = 'darkred', size = 15)